# MCP Historical Weather Comparison

## Overview

This notebook demonstrates how to use **Model Context Protocol (MCP)** to extend Claude's capabilities with real-time data access. We'll build a system that allows Claude to access historical weather data and compare annual weather statistics between two locations.

### What is MCP?

Model Context Protocol (MCP) is a standard that enables AI models to securely connect with external data sources and tools. Instead of being limited to training data, models can access live information, APIs, and services.

### What You'll Learn

1. **Limitations of LLMs without external tools** - See how Claude responds to weather queries without access to real data
2. **MCP Integration** - Connect Claude to a historical weather API
3. **Data Visualization** - Create aesthetically pleasing charts comparing weather patterns
4. **Interactive Analysis** - Ask Claude to analyze and compare weather data between locations

### Goals

By the end of this notebook, you'll have a working system that can:
- Fetch historical weather data for any location
- Compare annual weather statistics between two cities
- Generate visualizations and insights about weather patterns
- Demonstrate the power of extending LLMs with external data sources

## Setup

### Environment Setup

This notebook is designed to work in both Google Colab and local Jupyter environments. The following cell will automatically detect the environment and install the necessary dependencies.

In [1]:
import sys

# Check if we're running in Google Colab
IN_COLAB = 'google.colab' in sys.modules
print(f"Running in {'Google Colab' if IN_COLAB else 'local Jupyter environment'}")

# Install required packages using f-string with variables
if IN_COLAB:
    # Define packages that match pyproject.toml dependencies
    packages = [
        "anthropic>=0.66.0",
        "altair>=5.5.0", 
        "requests>=2.32.5"
    ]
    
    print("Installing required packages...")
    for package in packages:
        print(f"Installing {package}...")
        !pip install {package}
    
    print("✓ All packages installed successfully!")
else:
    print("Please ensure you have installed dependencies with:")
    print("  uv sync")
    print("or install manually with:")
    print("  pip install anthropic>=0.66.0 altair>=5.5.0 requests>=2.32.5")

Running in local Jupyter environment
Please ensure you have installed dependencies with:
  uv sync
or install manually with:
  pip install anthropic>=0.66.0 altair>=5.5.0 requests>=2.32.5


### Python Package Imports

Now let's import all the necessary packages for our weather comparison system.

In [2]:
import base64
from datetime import datetime, timedelta
import os
import json
from pprint import pprint

import getpass
from typing import Optional, Dict, Any

# Data handling and validation
import requests
from pydantic import BaseModel, Field, validator

# Visualization
import altair as alt
import vl_convert as vlc  # for converting Altair charts to images
alt.data_transformers.enable('json')

# For displaying rich notebook content
from IPython.display import Image, Markdown, display

# Claude API
import anthropic

import requests_cache  # for caching API responses
from retry_requests import retry  # for retrying API requests
import openmeteo_requests  # for making API requests to Open-Meteo

import numpy as np  # for numerical operations
import pandas as pd  # for data manipulation

print("All packages imported successfully!")

All packages imported successfully!


### Anthropic API Key Setup

We need to securely obtain your Anthropic API key to interact with Claude. This function will try multiple methods in order of preference for security.

In [3]:
def get_api_key():
    try:
        if IN_COLAB:
            # Import package for accessing user data
            from google.colab import userdata
            api_key = userdata.get('ANTHROPIC_API_KEY')
        else:
            api_key = os.environ.get("ANTHROPIC_API_KEY")
    except:
        # Prompt user for their API key
        api_key = getpass.getpass("Enter your Anthropic API key: ")
    return api_key

# Logging Flag

Let's also define a flag variable for controlling how verbose the output it.

In [4]:
verbose_output = True

## Motivation: Do we actually need to extend LLMs?

Before diving into MCP integration, let's first see what happens when we ask Claude to compare weather data between two locations **without giving it access to any external tools or data sources**.

This will demonstrate the fundamental limitation of LLMs: they can only work with information from their training data, which has a knowledge cutoff and may not include specific, current, or detailed data.

In [23]:
def ask_claude_without_tools(question: str) -> str:
    """
    Send a question to Claude without any external tools or data access.
    
    Args:
        question: The question to ask Claude
    
    Returns:
        Claude's response as a string
    """
    try:
        client = anthropic.Anthropic(api_key=get_api_key())
        message = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=1000,
            messages=[
                {
                    "role": "user", 
                    "content": question
                }
            ]
        )
        return message.content[0].text
    except Exception as e:
        return f"Error: {e}"

# Test Claude's response without external data
weather_question = """
Please compare the annual weather statistics for the years 2000-2023
for San Francisco, Redwood City, Seattle, and Austin. 
I'd like to see:

1. Days where the maximum temperature is between 18 and 24 degrees Celsius
2. Days where the precipitation exceeds 10 mm
3. Days where the sun duration exceeded 6*60*60 seconds
4. Days where the humidity is between 30 and 60%

Please provide specific data and create a comparison showing which city had more favorable weather conditions.
"""

print("🤖 Asking Claude about weather data WITHOUT external tools...")
print("="*70)
print(f"Question: {weather_question}")
print("="*70)
print("Claude's Response:")
print()

response = ask_claude_without_tools(weather_question)
display(Markdown(response))

🤖 Asking Claude about weather data WITHOUT external tools...
Question: 
Please compare the annual weather statistics for the years 2000-2023
for San Francisco, Redwood City, Seattle, and Austin. 
I'd like to see:

1. Days where the maximum temperature is between 18 and 24 degrees Celsius
2. Days where the precipitation exceeds 10 mm
3. Days where the sun duration exceeded 6*60*60 seconds
4. Days where the humidity is between 30 and 60%

Please provide specific data and create a comparison showing which city had more favorable weather conditions.

Claude's Response:



I don't have access to real-time weather databases or the ability to retrieve historical weather data for the specific analysis you're requesting. To get accurate annual weather statistics for San Francisco, Redwood City, Seattle, and Austin from 2000-2023, I'd recommend these reliable sources:

## Best Data Sources:

1. **NOAA Climate Data Online** (climate.gov)
   - Most comprehensive US weather data
   - Historical daily observations available

2. **Weather Underground Historical Section**
   - User-friendly interface for historical data
   - Good visualization tools

3. **OpenWeatherMap API**
   - Programmatic access to historical data
   - Good for bulk analysis

4. **National Weather Service Climate Data**
   - Official government source
   - Station-specific data available

## Analysis Framework:

For your specific metrics, you'd want to:

1. **Temperature 18-24°C (64-75°F)**: Count days in this "comfortable" range
2. **Precipitation >10mm (0.39 inches)**: Count "significant rain" days
3. **Sun duration >6 hours**: Count "sunny" days
4. **Humidity 30-60%**: Count "comfortable humidity" days

## Expected General Patterns:

- **San Francisco**: Mild temperatures year-round, moderate precipitation, frequent fog
- **Redwood City**: Similar to SF but slightly warmer and less fog
- **Seattle**: Cooler, much higher precipitation, especially in winter
- **Austin**: Hot summers, mild winters, variable precipitation

Would you like me to help you structure a data collection plan or provide code examples for analyzing weather data once you obtain it from these sources?

### Key Observations

As you can see from Claude's response above, without access to external data sources, the model has several limitations:

1. **No real-time data**: Claude can't access current or specific historical weather data
2. **General knowledge only**: Responses are based on general patterns from training data
3. **No specific metrics**: Can't provide exact precipitation amounts, temperatures, or day counts
4. **No visualizations**: Can't create charts or graphs from actual data

This demonstrates why **Model Context Protocol (MCP)** is valuable - it bridges the gap between the model's reasoning capabilities and real-world data access.

## Analysis

*This section will contain analysis of weather data patterns and comparisons between locations once we implement the MCP integration.*

```mermaid
graph LR
    A[Weather API] --> B[Data Analysis]
    B --> C[Visualization]
    C --> D[MCP Server]
    D --> E[Claude Client]
    E --> F[Happy User]
```

In this section we will create some Python functions that:
- retrieve daily data from the Open-Mateo Weather API for a list of locations
- aggregate daily data to annual summaries
- create a time-series chart, plotting the annual data and a trend

In [24]:
class Location(BaseModel):
    name: str
    longitude: float
    latitude: float

## Get weather data

In [25]:
api_url = "https://archive-api.open-meteo.com/v1/archive"

weather_variables = [
    "temperature_2m_max",
    "temperature_2m_mean",
    "temperature_2m_min",
    "relative_humidity_2m_max",
    "relative_humidity_2m_mean",
    "relative_humidity_2m_min",
    "rain_sum",
    "precipitation_hours",
    "sunshine_duration",
  ]

# Setup an Open-Meteo API client with a cache and retry mechanism
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

In [26]:
def get_weather_data(
      locations: list[Location],
      start_date: str,
      end_date: str,
      variables: list[str]
    ):
    """
    Get weather data for one or more locations.
    
    Args:
        locations: List of Location objects
        start_date: Start date in YYYY-MM-DD format
        end_date: End date in YYYY-MM-DD format
        variables: List of weather variables to retrieve
    
    Returns:
        Pandas DataFrame with weather data
    """
    
    def parse_response(variables, response, location_name):
        daily = response.Daily()
        daily_data_dict = {
            "date": pd.date_range(
            start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
            end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
            freq = pd.Timedelta(seconds = daily.Interval()),
            inclusive = "left"
            )
        }

        # Add the variable data.
        for i, variable in enumerate(variables):
            daily_data_dict[variable] = daily.Variables(i).ValuesAsNumpy()

        # Add a column for the location name.
        daily_data_dict["location_name"] = location_name

        return pd.DataFrame(daily_data_dict)

  
    params = {
        "latitude": [x.latitude for x in locations],
        "longitude": [x.longitude for x in locations],
        "start_date": start_date,
        "end_date": end_date,
        "daily": variables,
    }

    # Query for weather data and get one response per location.
    responses = openmeteo.weather_api(api_url, params=params)

    # Concatenate all of the responses into a single dataframe
    daily_df_list = [parse_response(variables, response, locations[i].name) for i, response in enumerate(responses)]
    daily_df = pd.concat(daily_df_list, axis=0)

    return daily_df

### Try it out

In [27]:
test_locations = [
  Location(
    name='San Francisco',
    latitude=37.7749,
    longitude=-122.4194,
  ),
  Location(
    name='Redwood City',
    latitude=37.4848,
    longitude=-122.2281,
  )
]

# Test the original functionality
daily_data = get_weather_data(
  locations=test_locations,
  start_date="2000-01-01",
  end_date="2019-12-31",
  variables=['temperature_2m_mean', 'temperature_2m_max', 'rain_sum', 'sunshine_duration']
)
daily_data

,date,temperature_2m_mean,temperature_2m_max,rain_sum,sunshine_duration,location_name
0,2000-01-01 00:00:00+00:00,7.517750,11.574000,0.0,29323.564453,San Francisco
1,2000-01-02 00:00:00+00:00,8.274000,12.074000,0.0,30698.628906,San Francisco
2,2000-01-03 00:00:00+00:00,7.753168,12.524000,0.0,30748.666016,San Francisco
3,2000-01-04 00:00:00+00:00,8.513582,11.974000,0.6,7515.774902,San Francisco
4,2000-01-05 00:00:00+00:00,10.132333,15.374000,0.0,25200.000000,San Francisco
...,...,...,...,...,...,...
7300,2019-12-27 00:00:00+00:00,9.049335,12.978499,0.0,30588.695312,Redwood City
7301,2019-12-28 00:00:00+00:00,8.543084,13.328500,0.0,30485.902344,Redwood City
7302,2019-12-29 00:00:00+00:00,10.351417,13.728499,3.6,12621.796875,Redwood City
7303,2019-12-30 00:00:00+00:00,10.066000,13.928500,5.9,25244.691406,Redwood City


## Calculate Annual Statistics

To enable better long-term comparisons, we can write a function that calculates annual statistics of how many times a variable exceeds a minimum or maximum threshold.

Examples:
- Days that the maximum temperature exceeds 30 degrees C
- Days that the mean temperatures is between 20 and 25 degrees C
- Days that rain exceeds 2 mm 

In [28]:
def calculate_annual_stats(
    daily_data: pd.DataFrame,
    variable: str,
    threshold_min: float = None,
    threshold_max: float = None
    ) -> pd.DataFrame:
  """
  Calculate annual statistics for the given daily data.
  
  Args:
    daily_data: DataFrame containing weather data
    variable: Name of the variable column to analyze
    threshold_min: Optional minimum threshold (inclusive)
    threshold_max: Optional maximum threshold (inclusive)
  
  Returns:
    DataFrame with columns: year, count, location_name
  """
  # Validate threshold parameters
  if threshold_min is None and threshold_max is None:
    raise ValueError("At least one of threshold_min or threshold_max must be provided")
  
  # Get unique location names from the daily data
  locations = daily_data['location_name'].unique()
  
  # Initialize list to store results
  results = []
  
  # Process each location
  for location in locations:
    # Filter data for this location and make an explicit copy to avoid SettingWithCopyWarning
    location_data = daily_data[daily_data['location_name'] == location].copy()
    
    # Extract year from date column
    location_data['year'] = pd.to_datetime(location_data['date']).dt.year
    
    # Apply threshold filters
    def count_days_in_range(x):
      mask = pd.Series(True, index=x.index)  # Start with all True
      
      if threshold_min is not None:
        mask = mask & (x >= threshold_min)
      
      if threshold_max is not None:
        mask = mask & (x <= threshold_max)
      
      return mask.sum()
    
    yearly_counts = location_data.groupby('year')[variable].apply(count_days_in_range)
    
    # Convert to dataframe
    yearly_df = pd.DataFrame({
      'year': yearly_counts.index,
      'count': yearly_counts.values,
      'location_name': location
    })
    
    results.append(yearly_df)
    
  # Combine all results
  return pd.concat(results, axis=0).reset_index(drop=True)

### Try it out

In [29]:
annual_stats = calculate_annual_stats(
    daily_data,
    variable='temperature_2m_max',
    threshold_min=20,
    threshold_max=25
)
annual_stats.head()

,year,count,location_name
0,2000,88,San Francisco
1,2001,101,San Francisco
2,2002,86,San Francisco
3,2003,106,San Francisco
4,2004,112,San Francisco


## Create a Timeseries Chart

Even tables of annual statistics can get pretty long, so we create a function for charting the data.

In [30]:
def create_annual_stats_chart(
    annual_stats: pd.DataFrame,
    title: str
  ) -> alt.Chart:
  """
  Create a chart showing the annual statistics.
  """
  # Base chart with data points
  base = alt.Chart(annual_stats).encode(
    x='year:O',
    y='count:Q',
    color='location_name:N'
  )

  # Create line chart
  lines = base.mark_line()

  # Add trend lines
  trend_lines = base.transform_regression(
    'year', 'count', 
    groupby=['location_name']
  ).mark_line(
    strokeDash=[5,5]
  ).encode(
    color='location_name:N'
  )

  # Combine the line and trend lines
  return (lines + trend_lines).properties(
    title=title
  )

### Try it out

In [31]:
create_annual_stats_chart(
    annual_stats,
    title='Days with Temperature Mean between 20 and 25 degrees C'
)

alt.LayerChart(...)

# MCP

Now that we have functions to retrieve, analyze, and format weather data, let's make it so we can query that information from an LLM. We will do this using the [Model Context Protocol (MCP)](https://modelcontextprotocol.io/). We start off by defining a Python class that access as a MCP client.

In [32]:
class MCPClient:
    """A client for interacting with Claude using the Model Context Protocol (MCP).
    
    This class handles registering tools, executing them, and managing conversations with 
    Claude using the MCP format. It processes both text and image outputs from tools
    and formats them appropriately for Claude's consumption.

    Attributes:
        client: The Anthropic client instance
        tools: List of registered MCP tools
        tool_functions: Dictionary mapping tool names to their implementation functions
    """
    def __init__(self, api_key):
        self.client = anthropic.Anthropic(api_key=api_key)
        self.tools = []
        self.tool_functions = {}
    
    def register_tool(self, name, description, input_schema, function):
        """Register an MCP tool"""
        tool = {
            "name": name,
            "description": description,
            "input_schema": input_schema
        }
        self.tools.append(tool)
        self.tool_functions[name] = function
    
    def execute_tool(self, tool_name, tool_input):
        """Execute a registered tool"""
        if tool_name in self.tool_functions:
            return self.tool_functions[tool_name](**tool_input)
        else:
            return f"Tool {tool_name} not found"
    
    def _process_mcp_tool_result(self, tool_result_json: str):
        """Process MCP tool result and return content for Claude"""
        try:
            result = json.loads(tool_result_json)
            if "error" in result:
                return [{"type": "text", "text": f"Error: {result['error']}"}]
            
            content = []
            
            # Add text content
            if "text" in result:
                content.append({
                    "type": "text",
                    "text": result["text"]
                })
            
            if "location" in result:
                content.append({
                    "type": "location",
                    "text": f"Location: {result['location']}"
                })

            # Add image content 
            if "chart_json" in result:
                content.append({
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "application/vnd.vegalite.v5+json",
                        "data": result["chart_json"]
                    }
                })
            return content
        
        except json.JSONDecodeError:
            # If not JSON, treat as plain text
            return [{"type": "text", "text": tool_result_json}]

    def chat_with_tools(self,
                        user_message,
                        model="claude-sonnet-4-20250514",
                        max_iterations=10):
        """Send messages and handle tool calls automatically"""
        
        # Initialize the messages list
        messages = [{"role": "user", "content": user_message}]

        # Track images generated by tools throughout the conversation
        tool_images = []

        for iteration in range(max_iterations):
            if verbose_output:
                display(Markdown(f"`chat_with_tools()`: Iteration {iteration + 1} --------------------------"))
            try:
                if verbose_output:
                    display(Markdown(f"`chat_with_tools()`: messages ="))
                    pprint(messages)
                # Make request to Claude with tools
                response = self.client.messages.create(
                    model=model,
                    max_tokens=4096,
                    tools=self.tools,
                    messages=messages
                )
                if verbose_output:
                    display(Markdown(f"`chat_with_tools()`: response.content ="))
                    for block in response.content:
                        pprint(block)
                
                # Add assistant's response to conversation
                messages.append({"role": "assistant", "content": response.content})

                if verbose_output:
                    display(Markdown(f'`chat_with_tools()`: {response.stop_reason = }'))
                if response.stop_reason == "tool_use":
                    # Process all tool uses via MCP
                    tool_results = []
                    
                    for block in response.content:
                        if block.type == "tool_use":

                            mcp_result = self.execute_tool(block.name, block.input)
                            content = self._process_mcp_tool_result(mcp_result)

                            # Extract and store any images from tool results
                            image_free_content = []
                            for content_item in content:
                                if content_item['type'] == 'image':
                                    tool_images.append(content_item)
                                else:
                                    image_free_content.append(content_item)

                            tool_results.append({
                                "type": "tool_result",
                                "tool_use_id": block.id,
                                "content": image_free_content
                            })
                    
                    # Add tool results to conversation
                    messages.append({"role": "user", "content": tool_results})
                    
                    # Continue conversation loop
                    continue
                    
                else:
                    # Final response - display both text and any tool-generated images
                    
                    # Create a custom response object that includes tool images
                    class ResponseWithImages:
                        def __init__(self, original_response, tool_images):
                            self.content = []
                            
                            # Add original text content
                            for block in original_response.content:
                                self.content.append(block)
                            
                            # Add tool-generated images as additional content
                            for image in tool_images:
                                # Convert tool image format to Claude response format
                                image_block = type('ImageBlock', (), {
                                    'type': 'image',
                                    'source': type('Source', (), {
                                        'data': image['source']['data'],
                                        'media_type': image['source']['media_type']
                                    })()
                                })()
                                self.content.append(image_block)
                    
                    # Return enhanced response with images
                    return ResponseWithImages(response, tool_images)
            
            except Exception as e:
                print(f"Error in iteration {iteration + 1}: {e}")
                return None
        
        print(f"\nReached maximum iterations ({max_iterations})")
        return None

Next we define an MCP tool for comparing two locations.

In [33]:
def compare_locations_mcp(
    location1: Location,
    location2: Location,
    variable: str,
    threshold_min: float = None,
    threshold_max: float = None,
    start_year: int = None,
    end_year: int = None
) -> str:
    """
    Compare the annual weather statistics for two locations.
    
    Args:
        location1: The first city
        location2: The second city  
        variable: Weather variable to compare (default: temperature_2m_max)
        threshold_min: Optional minimum threshold (inclusive)
        threshold_max: Optional maximum threshold (inclusive)
        start_year: Start year (default: 50 years before end_year)
        end_year: End year (default: previous year)
        
    Returns:
        JSON string containing weather comparison text and base64-encoded chart
    """
    try:
        if isinstance(location1, dict):
            location1 = Location(
                name=location1["name"],
                latitude=location1["latitude"],
                longitude=location1["longitude"]
            )
        if isinstance(location2, dict):
            location2 = Location(
                name=location2["name"],
                latitude=location2["latitude"],
                longitude=location2["longitude"]
            )

        # Set default year range if not provided
        if end_year is None:
            end_year = datetime.now().year - 1  # Default to previous year
        if start_year is None:
            start_year = end_year - 50  # Default to 50 years before end_year
        
        # Fetch weather data for both locations
        weather_data = get_weather_data(
            locations=[location1, location2],
            start_date=f"{start_year}-01-01",
            end_date=f"{end_year}-12-31",
            variables=[variable]
        )
                
        # Calculate annual statistics if thresholds were provided
        if threshold_min is not None or threshold_max is not None:
            annual_stats = calculate_annual_stats(
                daily_data=weather_data,
                variable=variable,
                threshold_min=threshold_min,
                threshold_max=threshold_max
            )
            
            # Create chart
            threshold_desc = ""
            if threshold_min is not None and threshold_max is not None:
                threshold_desc = f"Days of {variable} between {threshold_min} and {threshold_max}"
            elif threshold_min is not None:
                threshold_desc = f"Days of {variable} above {threshold_min}"
            elif threshold_max is not None:
                threshold_desc = f"Days of {variable} below {threshold_max}"
            
            chart = create_annual_stats_chart(annual_stats, threshold_desc)
        else:
            # Just show raw data comparison without thresholds
            annual_stats = None
            chart = None
        
        # Generate comparison text
        location1_data = weather_data[weather_data['location_name'] == location1.name]
        location2_data = weather_data[weather_data['location_name'] == location2.name]
        
        loc1_avg = location1_data[variable].mean()
        loc2_avg = location2_data[variable].mean()
        loc1_min = location1_data[variable].min() 
        loc1_max = location1_data[variable].max()
        loc2_min = location2_data[variable].min()
        loc2_max = location2_data[variable].max()
        
        comparison_text = f"""🌍 Location Comparison: {location1.name} vs {location2.name}
        
📊 {variable.replace('_', ' ').title()} Analysis:

🏙️ **{location1.name}**
• Average: {loc1_avg:.1f}
• Range: {loc1_min:.1f} to {loc1_max:.1f}
• Coordinates: {location1.latitude}, {location1.longitude}

🏙️ **{location2.name}**
• Average: {loc2_avg:.1f}  
• Range: {loc2_min:.1f} to {loc2_max:.1f}
• Coordinates: {location2.latitude}, {location2.longitude}
"""

        if annual_stats is not None:
            # Add threshold-based analysis
            loc1_stats = annual_stats[annual_stats['location_name'] == location1.name]
            loc2_stats = annual_stats[annual_stats['location_name'] == location2.name]
            
            loc1_avg_days = loc1_stats['count'].mean()
            loc2_avg_days = loc2_stats['count'].mean()
            
            comparison_text += f"""

🎯 **Threshold Analysis:**
• {location1.name}: {loc1_avg_days:.0f} days per year on average
• {location2.name}: {loc2_avg_days:.0f} days per year on average
• Criteria: {threshold_desc}"""

        result = {"text": comparison_text}
        if chart:
            result["chart_json"] = chart.to_dict()
            
        return json.dumps(result)
        
    except Exception as e:
        return json.dumps({"text": f"❌ Location comparison failed: {str(e)}"})

Initialize the Anthropic client

In [34]:
mcp_client = MCPClient(get_api_key())

Register the tool with the MCP Client.

In [35]:
mcp_client.register_tool(
    name="compare_locations_mcp",
    description="Compare annual weather statistics between two cities with optional threshold analysis and visualization.",
    input_schema={
            "type": "object",
            "properties": {
                "location1": {
                    "type": "object",
                    "description": "First location to compare",
                    "properties": {
                        "name": {"type": "string"},
                        "latitude": {"type": "number"},
                        "longitude": {"type": "number"}
                    },
                    "required": ["name", "latitude", "longitude"]
                },
                "location2": {
                    "type": "object",
                    "description": "Second location to compare", 
                    "properties": {
                        "name": {"type": "string"},
                        "latitude": {"type": "number"},
                        "longitude": {"type": "number"}
                    },
                    "required": ["name", "latitude", "longitude"]
                },
                "variable": {
                    "type": "string",
                    "description": "Weather variable to compare (default: temperature_2m_max)",
                    "enum": weather_variables,
                },
                "threshold_min": {
                    "type": "number",
                    "description": "Optional minimum threshold for counting days (inclusive)"
                },
                "threshold_max": {
                    "type": "number", 
                    "description": "Optional maximum threshold for counting days (inclusive)"
                },
                "start_year": {
                    "type": "integer",
                    "description": "Start year (default: 10 years before end_year)"
                },
                "end_year": {
                    "type": "integer",
                    "description": "End year (default: previous year)"
                }
            },
            "required": ["location1", "location2"]
        },
    function=compare_locations_mcp
)

## Try it out

We can verify that the MCP tools is configured properly, by executing the tool directly.

In [36]:
tool_results_str = mcp_client.execute_tool(
  tool_name="compare_locations_mcp",
  tool_input={
    "location1": {
        "name": "San Francisco",
        "latitude": 37.7749,
        "longitude": -122.4194
    },
    "location2": {
        "name": "San Jose", 
        "latitude": 37.3382,
        "longitude": -121.8863
    },
    "variable": "temperature_2m_max",
    "threshold_min": 25,  # Count days above 25°C
    "start_year": 2000,
    "end_year": 2019
})

tool_results = json.loads(tool_results_str)
display(Markdown(tool_results['text']))
if 'chart_json' in tool_results:
  display(alt.Chart.from_dict(tool_results['chart_json']))

🌍 Location Comparison: San Francisco vs San Jose
        
📊 Temperature 2M Max Analysis:

🏙️ **San Francisco**
• Average: 18.3
• Range: 6.9 to 38.4
• Coordinates: 37.7749, -122.4194

🏙️ **San Jose**
• Average: 21.9  
• Range: 7.8 to 43.3
• Coordinates: 37.3382, -121.8863


🎯 **Threshold Analysis:**
• San Francisco: 25 days per year on average
• San Jose: 125 days per year on average
• Criteria: Days of temperature_2m_max above 25

alt.LayerChart(...)

## Helper function

Finally we create a helper function that converts the MCP Client response data into representations that the Jupyter notebook can display.

In [37]:
def display_claude_response(response):
    """Display Claude response with both text and images"""    
    if response:
        for content in response.content:
            if content.type == "text":
                display(Markdown(content.text))
            elif content.type == "image":
                if content.source.media_type == "application/vnd.vegalite.v5+json":
                    # Recreate the chart from JSON dictionary
                    altair_chart = alt.Chart.from_dict(content.source.data)
                    display(altair_chart)


# Examples

In [38]:
print(weather_question)


Please compare the annual weather statistics for the years 2000-2023
for San Francisco, Redwood City, Seattle, and Austin. 
I'd like to see:

1. Days where the maximum temperature is between 18 and 24 degrees Celsius
2. Days where the precipitation exceeds 10 mm
3. Days where the sun duration exceeded 6*60*60 seconds
4. Days where the humidity is between 30 and 60%

Please provide specific data and create a comparison showing which city had more favorable weather conditions.



In [39]:
response = mcp_client.chat_with_tools(weather_question)

`chat_with_tools()`: Iteration 1 --------------------------

`chat_with_tools()`: messages =

[{'content': '\n'
             'Please compare the annual weather statistics for the years '
             '2000-2023\n'
             'for San Francisco, Redwood City, Seattle, and Austin. \n'
             "I'd like to see:\n"
             '\n'
             '1. Days where the maximum temperature is between 18 and 24 '
             'degrees Celsius\n'
             '2. Days where the precipitation exceeds 10 mm\n'
             '3. Days where the sun duration exceeded 6*60*60 seconds\n'
             '4. Days where the humidity is between 30 and 60%\n'
             '\n'
             'Please provide specific data and create a comparison showing '
             'which city had more favorable weather conditions.\n',
  'role': 'user'}]


`chat_with_tools()`: response.content =

TextBlock(citations=None, text="I'll help you compare the weather statistics for these four cities. Since the comparison function works with two locations at a time, I'll need to make multiple comparisons to cover all the cities and weather variables you're interested in.\n\nLet me start by comparing San Francisco with each of the other cities for all the requested weather conditions:", type='text')
ToolUseBlock(id='toolu_017kU3tB7TGsCnfSFdHWDRyt', input={'location1': {'name': 'San Francisco', 'latitude': 37.7749, 'longitude': -122.4194}, 'location2': {'name': 'Redwood City', 'latitude': 37.4852, 'longitude': -122.2364}, 'start_year': 2000, 'end_year': 2023, 'variable': 'temperature_2m_max', 'threshold_min': 18, 'threshold_max': 24}, name='compare_locations_mcp', type='tool_use')
ToolUseBlock(id='toolu_01WhnoZcJu31FkRwaDFKmb4k', input={'location1': {'name': 'San Francisco', 'latitude': 37.7749, 'longitude': -122.4194}, 'location2': {'name': 'Seattle', 'latitude': 47.6062, 'longitude': 

...

`chat_with_tools()`: messages =

[{'content': '\n'
             'Please compare the annual weather statistics for the years '
             '2000-2023\n'
             'for San Francisco, Redwood City, Seattle, and Austin. \n'
             "I'd like to see:\n"
             '\n'
             '1. Days where the maximum temperature is between 18 and 24 '
             'degrees Celsius\n'
             '2. Days where the precipitation exceeds 10 mm\n'
             '3. Days where the sun duration exceeded 6*60*60 seconds\n'
             '4. Days where the humidity is between 30 and 60%\n'
             '\n'
             'Please provide specific data and create a comparison showing '
             'which city had more favorable weather conditions.\n',
  'role': 'user'},
 {'content': [TextBlock(citations=None, text="I'll help you compare the weather statistics for these four cities. Since the comparison function works with two locations at a time, I'll need to make multiple comparisons to cover all the cities and weather variables

`chat_with_tools()`: response.content =

TextBlock(citations=None, text="Due to API rate limits, I've been able to gather partial data. Let me provide you with the analysis based on what I've collected, and we can get the remaining data in a follow-up:\n\n## Weather Comparison Analysis (2000-2023)\n\n### 🌡️ **Temperature Analysis: Days with Max Temperature 18-24°C**\n\nBased on the data I've collected:\n\n| City | Average Days per Year (18-24°C) |\n|------|----------------------------------|\n| **San Francisco** | **138 days** ⭐ |\n| Redwood City | 130 days |\n| Seattle | 80 days |\n| Austin | 73 days |\n\n**Winner: San Francisco** - Has the most days with comfortable temperatures in the 18-24°C range.\n\n### 🌧️ **Precipitation Analysis: Days with >10mm Rain**\n\nFrom the available data:\n\n| City | Average Days per Year (>10mm) |\n|------|-------------------------------|\n| San Francisco | 21 days |\n| Redwood City | 17 days ⭐ |\n\n**Partial Winner: Redwood City** - Fewer heavy rain days means better weather for most activit

`chat_with_tools()`: response.stop_reason = 'end_turn'

In [40]:
display_claude_response(response)

Due to API rate limits, I've been able to gather partial data. Let me provide you with the analysis based on what I've collected, and we can get the remaining data in a follow-up:

## Weather Comparison Analysis (2000-2023)

### 🌡️ **Temperature Analysis: Days with Max Temperature 18-24°C**

Based on the data I've collected:

| City | Average Days per Year (18-24°C) |
|------|----------------------------------|
| **San Francisco** | **138 days** ⭐ |
| Redwood City | 130 days |
| Seattle | 80 days |
| Austin | 73 days |

**Winner: San Francisco** - Has the most days with comfortable temperatures in the 18-24°C range.

### 🌧️ **Precipitation Analysis: Days with >10mm Rain**

From the available data:

| City | Average Days per Year (>10mm) |
|------|-------------------------------|
| San Francisco | 21 days |
| Redwood City | 17 days ⭐ |

**Partial Winner: Redwood City** - Fewer heavy rain days means better weather for most activities.

### **Key Findings So Far:**

1. **San Francisco** excels at having the most days with ideal temperatures (18-24°C), averaging 138 days per year
2. **Redwood City** has fewer heavy rain days compared to San Francisco
3. **Seattle** has significantly fewer comfortable temperature days (only 80 per year)
4. **Austin** surprisingly has the fewest comfortable temperature days (73 per year), likely due to it being too hot most of the time

To complete the analysis, I would need to gather the remaining data for:
- Precipitation comparison for Seattle and Austin
- Sunshine duration analysis for all cities (>6 hours per day)
- Humidity analysis for all cities (30-60% range)

Would you like me to continue with the remaining comparisons? Due to API rate limits, I may need to make the requests in smaller batches.

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)

alt.LayerChart(...)